In [1]:
import torch
from torchvision import transforms
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd


import random

# 读取从字符到编码的索引

In [2]:
embed =102
charset=np.load("charset.npy",allow_pickle=True).item()
char_to_int =  np.load("char_to_int.npy",allow_pickle=True).item()
print(charset)
print(f"len of charset: {len(charset)}")

{'\\', '+', '7', '3', '5', 'M', 'E', '/', 'N', 'F', 'O', ']', '!', 'r', '1', 'g', '(', '@', '=', '4', 'C', '-', '[', 'B', 'H', 'S', '6', '*', '#', ')', 'l', '2'}
len of charset: 32


# 读取数据和标签

In [3]:
assay1 = "BRD4-BD1" 
train_df = pd.read_csv("train_BRD4-BD1.csv")
test_df =pd.read_csv("test_BRD4-BD1.csv")
X_train_smiles = np.array(list(train_df["SMILES"]))

Y_train = train_df[assay1].values
Y_train = np.array(Y_train)

print(X_train_smiles[0:5],type(X_train_smiles))


['NS(=O)(=O)C1=CC=C(NC2=NC3=C(C=CC=C3)C(C3=CC=C(Cl)C=C3)=N2)C=C1'
 'NS(=O)(=O)C1=CC=CC=C1NC1=NC2=C(C=CC=C2)C(C2=CC=C(Cl)C=C2)=N1'
 '[H]C1=CC=C2N=C(NC3=CC=C(S(N)(=O)=O)C=C3)N=C(C3=CC=C(Br)C=C3)C2=C1'
 '[H]C1=CC=C2N=C(NC3=CC=C(S(N)(=O)=O)C=C3)N=C(C3=CC=C(F)C=C3)C2=C1'
 '[H]C1=CC=C2N=C(NC3=CC=CC=C3S(N)(=O)=O)N=C(C3=CC=C(F)C=C3)C2=C1'] <class 'numpy.ndarray'>


# 为数据生成one-hot 编码

In [4]:
def vectorize(smiles):
        one_hot =  np.zeros((smiles.shape[0], embed , len(charset)),dtype=np.int8)
        # 为所有数据生成形状为（embed，len(charset))的onehot编码,
        for i,smile in enumerate(smiles): 
            #encode the start char 开始编码为！
            one_hot[i,0,char_to_int["!"]] = 1
            #encode the rest of the chars
            for j,c in enumerate(smile):
                one_hot[i,j+1,char_to_int[c]] = 1
            #Encode endchar 结束编码为 E
            one_hot[i,len(smile)+1:,char_to_int["E"]] = 1
        #Return two, one for input and the other for output
        return one_hot

In [5]:
X_train = torch.Tensor(vectorize(X_train_smiles)).unsqueeze(1)
Y_train= torch.Tensor(Y_train)
# X_test = vectorize(X_test)
print(X_train[0].shape)
print(X_train[0])
print(np.argmax(X_train[0],axis=1)) # 按行编码

torch.Size([1, 102, 32])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
tensor([[ 0,  0,  0,  0,  3,  0,  2,  0, 63,  0,  4,  0, 12,  0,  0,  6,  0,  0,
         48, 25,  0,  5,  0,  0,  0,  1,  0,  0,  0, 11, 21,  0]])


In [6]:
def data_iter(batch_size,features,labels):
    num_examples=len(features)
    indices=list(range(num_examples))
    random.shuffle(indices)
    for i in range(0,num_examples,batch_size):
        batch_indices=torch.tensor(
            indices[i:min(i+batch_size,num_examples)]
        )
        yield features[batch_indices],labels[batch_indices]

In [7]:
class Residual(nn.Module):  #@save
    """The Residual block of ResNet."""
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk


In [8]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))

In [9]:
net = nn.Sequential(b1, b2, b3, b4,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(256, 1))

In [10]:
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.001,weight_decay=0.01)

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Residual(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-

In [12]:
num_epochs =80
batch_size=10
loss_list=[]
def train(batch_size,train_x,train_y):
    for epoch in range(num_epochs):
        if isinstance(net, torch.nn.Module):
            net.train()
        for X, y in data_iter(batch_size,train_x,train_y):
            X,y=X.to(device),y.to(device)
            l = loss(net(X) ,y)
            trainer.zero_grad()
            l.backward()
            trainer.step()
        train_x,train_y=train_x.to(device),train_y.to(device)
        l=loss(net(train_x),train_y)
        loss_list.append(l)
        # if (epoch+1)%1==1:
        print(f'epoch {epoch}, loss {l:f}')
         
       
        
train(batch_size,X_train,Y_train)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([185])) that is different to the input size (torch.Size([185, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, ta

epoch 0, loss 65.602699
epoch 1, loss 65.425217
epoch 2, loss 65.829826
epoch 3, loss 65.504189
epoch 4, loss 66.023621
epoch 5, loss 65.357567
epoch 6, loss 66.419868
epoch 7, loss 65.429329
epoch 8, loss 65.630821
epoch 9, loss 65.435165
epoch 10, loss 65.314713
epoch 11, loss 65.482155
epoch 12, loss 70.251419
epoch 13, loss 66.204926
epoch 14, loss 65.421852
epoch 15, loss 65.323616
epoch 16, loss 65.973892
epoch 17, loss 65.590546
epoch 18, loss 66.977615
epoch 19, loss 65.325439
epoch 20, loss 65.810349
epoch 21, loss 65.573059
epoch 22, loss 66.160469
epoch 23, loss 66.004585
epoch 24, loss 65.359047
epoch 25, loss 65.524307
epoch 26, loss 65.598656
epoch 27, loss 65.516022
epoch 28, loss 65.331261
epoch 29, loss 65.320190
epoch 30, loss 66.067284
epoch 31, loss 67.161888
epoch 32, loss 66.382507
epoch 33, loss 65.348656
epoch 34, loss 69.510475
epoch 35, loss 66.146858
epoch 36, loss 66.261452
epoch 37, loss 66.536919
epoch 38, loss 65.697136
epoch 39, loss 65.883438
epoch 40, 

RuntimeError: CUDA out of memory. Tried to allocate 38.00 MiB (GPU 0; 11.77 GiB total capacity; 9.77 GiB already allocated; 26.06 MiB free; 10.05 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF